In [1]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    Source_URL : str
    local_data_file : Path
    unzip_dir : Path



In [2]:
from CNNClassifier.utils.utils import read_yaml,create_directory
from CNNClassifier.constants import *


In [4]:
class ConfigurationManager:

    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])
        

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion
        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            Source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir

            )
        return data_ingestion_config
        


In [5]:
import os
os.chdir("../")
import urllib.request as request
from zipfile import ZipFile

In [6]:
from tqdm import tqdm
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            
            request.urlretrieve(
                url = self.config.Source_URL,
                filename = self.config.local_data_file

            )
       

    def get_updated_list_of_files(self,list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg")]

    def preprocess(self,zf,f,working_dir):
        target_filepath = os.path.join(working_dir,f)
        if not os.path.exists(target_filepath):
            zf.extract(f,working_dir)

    def unzip_and_clean(self):
        with ZipFile(file = self.config.local_data_file, mode = "r") as zf:
            list_of_file = zf.namelist()
            updated_list_of_file = self.get_updated_list_of_files(list_of_file)
            for f in tqdm(updated_list_of_file):
                self.preprocess(zf, f, self.config.unzip_dir)


In [7]:
%pwd

'/home/eliyas/Desktop/12333/CNN-Classifier'

In [8]:
config = ConfigurationManager()

[2023-11-21 15:13:07,337:INFO:utils]:Created directory at :artifacts


In [9]:


config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()

data_ingestion = DataIngestion(config = data_ingestion_config)

data_ingestion.download_file()

data_ingestion.unzip_and_clean()


[2023-11-21 15:18:15,451:INFO:utils]:Created directory at :artifacts
[2023-11-21 15:18:15,455:INFO:utils]:Created directory at :artifacts/data_ingestion


100%|██████████| 25000/25000 [00:00<00:00, 47390.25it/s]
